# Sign language recognition using CNN

* This Notebook inspirde by the course Scientific computation in Python from IDC and Dr. Yoav Ram

In this Exercise we would train a CNN to clasify images of sign languge.
This Notebook has 4 main parts:

    1. Loading the Data.
    2. Prepering the data for the model(images and labels).
    3. Creating and training the model.
    4. Presinting the results.

First lets do some imports

In [ ]:
import os
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import keras
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.layers.normalization import BatchNormalization

The First step would be loading the data from the Dataset folder, we load all the images. 
We also load the data from the csv files which would help us to crop the images.

In [ ]:
def getFileName(path):
    return (path.split("../input/sign-language-images/Dataset/")[1])

In [ ]:
images = {}
metaData = []
folders = next(os.walk('../input/sign-language-images/Dataset'))[1]
for folder in folders:
    csvFile = pd.read_csv("../input/sign-language-images/Dataset/%s/%s_loc.csv" %(folder,folder))
    metaData.append(np.array(csvFile))
    for img in glob.glob("../input/sign-language-images/Dataset/%s/*.jpg" %folder):
        n= mpimg.imread(img)
        picName = getFileName(img)
        images.update({picName : n})

After loading the data we crop the images so we would have only the "interesting" part of the images, and resize all the images to the same size(keras requirment).

In [ ]:
def preprocess(image,box):
    return cv2.resize(image[box[0]:box[1],box[2]:box[3]], (90,90))

In [ ]:
imagesAfterCrop = {}
for userImages in metaData:
    for img in userImages:
        imagesAfterCrop.update({img[0] : preprocess(images[img[0]],[img[2], img[4], img[1], img[3]])})

In [ ]:
img=imagesAfterCrop["user_10/A1.jpg"]
imgplot = plt.imshow(img)
plt.show()
imagesAfterCrop["user_10/A1.jpg"].shape

Here we create the labels array, using the pictures names.
The labels are one hot encoding, we would make sure everything fine by printing the shape of the labels array.

In [ ]:
def createLabelsArray(images):
    counter = 0;
    labels = np.zeros((len(images),26))
    for img in images:
        labels[counter][(ord(img.split('/')[1][0]) - ord('A'))] = 1 
        counter = counter + 1
    return labels
    
        
y = createLabelsArray(imagesAfterCrop)
print(y.shape)

We almost there!
Now we split the data to train and test, and rescale all the x(pixels) values to be between 0 to 1(keras requirment)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(list(imagesAfterCrop.values()), y, test_size=0.2, random_state = 0)
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train / 255
x_test = x_test / 255

Finaly we can create and train our model!
for better results we use BatchNormalization and the Adam optimizer.
We use the softmax function for the classification.

In [ ]:
num_classes = 26

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), input_shape=(90, 90, 3)))
model.add(keras.layers.Activation('relu'))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3)))
model.add(keras.layers.Activation('relu'))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3)))
model.add(keras.layers.Activation('relu'))
model.add(BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024))
model.add(keras.layers.Activation('relu'))
model.add(BatchNormalization())
model.add(keras.layers.Dense(num_classes))
model.add(keras.layers.Activation('softmax'))

In [ ]:
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

history = model.fit(
    x_train, y_train,
    batch_size=16,
    epochs=10,
    validation_data=(x_test, y_test)
).history

In [ ]:
# Score trained model.
loss, acc = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', acc)

Great!
we get ~90% accuracy on the test set.
here are some examples of our predictions.

In [ ]:
def decodeReal(yHat):
    return chr(np.argmax(yHat) + 97).upper()

def decodePrediction(number):
    return chr(number + 97).upper()

In [ ]:
fig=plt.figure(figsize=(15, 15))
columns = 3
rows = 3
for i in range(1, columns*rows +1):
    img = x_test[0+ i]
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.axis('off')
    print("The predicted Letter for Example#"+str(i)+" is "+decodePrediction(model.predict_classes(np.array([x_test[i],]))))
    print("The real Letter for Example#"+str(i)+ " is " +decodeReal(y_test[i]))
plt.show()

In [ ]:
allLettersTestArray = np.zeros(26)
allLettersTestMistakesArray = np.zeros(26)
counter = 0
for example in x_test:
    yhat = np.argmax(y_test[counter])
    allLettersTestArray[yhat] += 1
    if(model.predict_classes(np.array([x_test[counter],])) != yhat):
        allLettersTestMistakesArray[yhat] += 1
    counter += 1
allLettersTestArray[allLettersTestArray == 0] = -1    
mistakesPerLetter = allLettersTestMistakesArray / allLettersTestArray
lettersArray = np.array(list(map(chr, range(97, 123))))
df = pd.DataFrame({'leters' : lettersArray, 'mistakes' : mistakesPerLetter})
df['colors'] = 'b'
df.loc[df.mistakes > 0.15, 'colors'] ='r'

In [ ]:
f, ax = plt.subplots(figsize=(18,5))
plt.bar(df.leters, df.mistakes, color=df.colors)
ax.set_title('Mistake Frequency by letter', fontsize=18)
ax.set_ylabel('Mistake Frequency', fontsize=18)

Here we can see the Mistakes frequncy per letter,
the red ones are the letters which had the most mistakes.
There could be many reasons why we had more mistakes on those letters(maybe similarity to other letters), but thats out of our scope :)

# Thank you for reading